In [ ]:
from transformers import RobertaTokenizerFast, RobertaForTokenClassification, BatchEncoding, CharSpan
from datasets import load_dataset, Dataset
from tqdm.notebook import tqdm
import torch, os, numpy as np

import utils

import importlib
importlib.reload(utils);

device = "cuda" if torch.cuda.is_available else "cpu"

In [ ]:
path = 'checkpoints/roberta-training-default-dataset-fp16/checkpoint-1005'

tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base', add_prefix_space=True)
model = RobertaForTokenClassification.from_pretrained(path, num_labels=37).to(device)

In [ ]:
ds = load_dataset("tner/ontonotes5", keep_in_memory=True, num_proc=os.cpu_count())

validation_tokenized_ds: Dataset = utils.process_dataset(ds['test'], tokenizer)

In [ ]:
batch_size = 4
total_ds_len = len(validation_tokenized_ds)

all_predictions = np.zeros((4*total_ds_len, 512), dtype=int)
all_labels = np.zeros((4*total_ds_len, 512), dtype=int)

with torch.no_grad():
    for idx in tqdm(range(0, total_ds_len, batch_size)):
        
        input_ids = torch.tensor(validation_tokenized_ds['input_ids'][idx:idx + batch_size]).to(device)
        attention_mask = torch.tensor(validation_tokenized_ds['attention_mask'][idx:idx + batch_size]).to(device)

        logits = model(input_ids, attention_mask).logits
        predicted_token_class_ids = logits.argmax(-1).to("cpu").numpy()

        all_predictions[idx:idx + batch_size] = predicted_token_class_ids

        labels = np.array(validation_tokenized_ds['labels'][idx:idx + batch_size])
        all_labels[idx:idx + batch_size] = labels

print(utils.compute_accuracy(all_predictions, all_labels))